<div style="display:block">
    <div style="width: 20%; display: inline-block; text-align: left;">
        <img src="http://upload.wikimedia.org/wikipedia/en/0/0c/Mu_Sigma_Logo.jpg" style="height:75px; margin-left:0px" />
    </div>
    <div style="width: 59%; display: inline-block">
        <h1  style="text-align: center"> Social Media Analytics - Treatment pattern Identification </h1><br>
        <div style="width: 100%; text-align: center; display: inline-block;"><i>Author:</i> <strong>Varun Behl</strong> </div>
    </div>
    <div style="width: 20%; text-align: right; display: inline-block;">
        <div style="width: 100%; text-align: left; display: inline-block;">
            <i>Created: </i>
            <time datetime="2018-02-14" pubdate>13th Feb 2018</time>
            <i>Modified: </i>
            <time datetime="2018-02-21" pubdate>21st Feb 2018</time>
        </div>
    </div>
</div>

# Objective

The aim of this study is to identify the treatment pattern of patients (suffering or cured) from melanoma, based on the data scraped through different forums related to melanoma.The scraped data consists of comments of patients and their friends and family.

### Phase 1:
The first step is to predict whether the post discusses systemic therapy or not. This is done with a SVM clasification algorithm, which is most appropriate for text classification. 


In [1]:
# Importing necassary packages 
import string
import pandas as pd
import numpy as np
import nltk
import re

# Sklearn package to import necessary vectorizing algorithms
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer

#NLTK package for text mining operations 
from nltk.corpus import stopwords as sw
from nltk.corpus import wordnet as wn
from nltk import wordpunct_tokenize
from nltk import WordNetLemmatizer
from nltk import sent_tokenize
from nltk.tokenize import sent_tokenize
from nltk import pos_tag

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report as clsr
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
import pickle
from sklearn.cross_validation import KFold
from nltk.tokenize import sent_tokenize
from __future__ import division
from scipy import sparse , vstack , hstack
from sklearn.svm import SVC
from sklearn.utils import check_random_state, safe_indexing

C:\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Step 1 : Importing social media dataset

##### Step 1a : Data scraped and tokenized to sentence level
The imported data set consists of posts which have been tokenized to sentence level with a corresponding column having flags 1 and 0: 1 (talking about drug consumption), and 0 (no mention of drug or treatment)

In [7]:
# Reading the sentenced tokenized dataset
posts = pd.read_csv('SMA_collated_tagged_posts.csv')

In [2]:
posts_test = pd.read_csv('Final_data_sentence_level.csv')

In [114]:

posts.head(5)


Docs  unique_key                                Tokenized.Sentences  \
0     6       59473  ?�????  Clearly, any drug with more than a 30%...   
1     6       59473  ?�????  Again, hopefully you will need none of...   
2     6       59473  ?�????  Nonetheless, the...data were impressiv...   
3     6       59473  There are three here in the US...the BMS produ...   
4     6       59473          ?�????  Anti-PD1 drugs are immunotherapy.   

   Old_flag  New_flag  
0         1         0  
1         1         0  
2         1         0  
3         1         0  
4         1         0

### Step 2 :  Pre-processing text columns

Textual data has to be processed before incorporating it into the model. Various text mining operations are done on the textual data - like cleaning html tags, special characters, stripping URL tags, etc. Word tokens are created at the end of this process, which will be the independent features for the model. 


In [3]:
## Pre-processing the text column

lemmatizer = WordNetLemmatizer()

def identity(arg):
    """
    Simple identity function works as a passthrough.
    """
    return arg

## Removing HTML tags
def cleanhtml(raw_html):
    cleanr = re.compile(r'<[^>]+>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

## Removing URL
def strip_URL(text):
    return re.sub(r'(https|http)?:\/\/(\w|\.|-|\/|\?|\=|\&|\%)*\b', '', text, flags=re.MULTILINE)

## Removing Hashtags and '@' tags 
def strip_all_tags(text):
    entity_prefixes = ['@','#','[]']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

## Removing special characters 
def strip_special_chars(text):
    return re.sub('\W+',' ', text)

## Removing extra white spaces
def strip_extra_white_spaces(text):
    return re.sub(' +',' ',text).strip()

## Removing extra new lines
def strip_extra_new_lines(text):
    return re.sub("\n+","\n", text)

def inverse_transform(l):
        sent = ''
        for word in l:
            sent += word+" "
        return [sent]


def tokenize(document):
    document = cleanhtml(document)
    document = strip_URL(document)
    document = strip_all_tags(document)
    document = strip_special_chars(document)
    document = strip_extra_white_spaces(document)
    document = strip_extra_new_lines(document)
    stopwords_custom = pd.read_csv("stopwords1.csv")
    stopwords_custom = stopwords_custom.ix[:,0].map(lambda x:x.lower())
        
    #Breaking the document into sentences
    for sent in sent_tokenize(document):
        # Breaking the sentence into parts of speech-tagged tokens
        for token, tag in pos_tag(wordpunct_tokenize(sent)):
            # Applying pre-processing to the token
            token = token.lower() 

            #If there is a stopword, ignore the token and continue
            if token.lower in stopwords_custom.tolist():
                continue

            # If there is a punctuation, ignore the token and continue
            if all(char in string.punctuation for char in token):
                  continue
            
            yield token

#Lemmatize the token and yield
            #lemma = lemmatize(token, tag)
            #yield lemma

def lemmatize(token, tag):
    tag = {
       'N': wn.NOUN,
       'V': wn.VERB,
       'R': wn.ADV,
       'J': wn.ADJ
    }.get(tag[0], wn.NOUN)

    #return lemmatizer.lemmatize(token, tag)
    
def preprocessing(X):
    #tokens_dict = {}
    tokens_list = []
    count = 0
    for doc in X:
        tokens = list(tokenize(doc))
        # tokens_list
        tokens_list.append(tokens)
#tokens_dict[count] = inverse_transform(tokens) count += 1
    return tokens_list


#### Step 2a : Pre-processing and creating a list of word tokens

In [8]:
#pre-processed data :List of words (non-string)
words= preprocessing(posts['Tokenized.Sentences'])
#output type is a list of words ,here words are not in string format


In [4]:
Words_test = preprocessing(posts_test['Tokenized.Sentences'])

In [116]:
# Word Tokens are created after preprocessing of the textual data
print words[0]
print type(words)

['clearly', 'any', 'drug', 'with', 'more', 'than', 'a', '30', 'response', 'rate', 'in', 'patients', 'who', 'have', 'been', 'through', 'a', 'median', 'of', '3', 'previous', 'regimens', 'in', 'melanoma', 'is', 'looking', 'very', 'good']
<type 'list'>


#### Step 2b : Creating a list of strings
For further operation with tfidf, input should be a string of words/phrase. The following code will create strings of phrases which can be appended to a list


In [9]:
#List of strings -in phrases -ready 
List_strings =[] 
for i in words:
    a = " ".join(str(x) for x in i)
    List_strings.append(a) 

In [5]:
List_strings_test =[] 
for i in Words_test:
    a = " ".join(str(x) for x in i)
    List_strings_test.append(a)

In [118]:
print List_strings[0]
print List_strings[1]

clearly any drug with more than a 30 response rate in patients who have been through a median of 3 previous regimens in melanoma is looking very good
again hopefully you will need none of these


In [ ]:
posts['Post'] = List

## Step 3 : Feature Generation
### Vectorizing
For feature generation, we need to create a matrix of vectorized features. For converting a collection of raw documents to a matrix of features, we use TF-IDF vectorizer (Term Frequency – Inverse Document Frequency). It assigns scores/weights to each word, which is considered as a data point for the model.

<b>Term Frequency: This summarizes how often a given word appears within a document.</b><br>
<b>Inverse Document Frequency: This downscales words that appear a lot across documents.</b><br>
TF-IDF are word frequency scores that try to highlight words that are more interesting i.e. frequent in a document, but not across documents.

The Tfidf Vectorizer will tokenize documents, learn the vocabulary and inverse document frequency weightings, and allows to encode new documents.

The dependent variable in the dataset has skewness which has to be eliminated before passing it through the model. 
The appended list of strings is passed through tfidf-vectorizer which a sparse csr matrix  .The generated matrix are used as features for the dataset ,with values being the tfidf score


In [10]:
# Dependent variable is defined
y = posts["New_flag"]
subsample_size = 1.0

In [16]:
vectorizer = TfidfVectorizer(use_idf= True)
#vectorizer= CountVectorizer()
X_1 = vectorizer.fit_transform(List_strings)
#Vectorized output is in csr matrix format



### Step 4 : Training and testing split
For classification model , the required training and testing dataset can be prepared with tts function . Output for features and and data point is a sparse matrix , where as the dependent variable is numpy array


In [12]:
from sklearn.cross_validation import train_test_split as tts
from scipy import sparse

#Training and testing split - with a split of 70:30
X_train, X_test, y_train, y_test = tts(X_1,y, test_size=0.3, random_state=123)


### Step 5 : Balancing using Synthetic Minority Over-sampling Technique

This technique is followed to avoid overfitting which occurs when exact replicas of minority instances are added to the main dataset. A subset of data is taken from the minority class as an example and then new synthetic similar instances are created. These synthetic instances are then added to the original dataset. The new dataset is used as a sample to train the classification models.

SMOTE is used to create a balanced dataset which can be passed through the best-fit classifier model. The random state or seed should be equal to the assigned for the selected model

Input can be a spicy sparse matrix or numpy array


In [13]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=123, ratio = 0.8)
x_res, y_res = sm.fit_sample(X_train,y_train)

C:\Anaconda2\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)


### Running Linear SVM model 
Support vector mahcine is a supervised learning method used for classification, regression and outliers detection. It is memory efficient as it uses a subset of training points in the decision function (called support vectors) . It works good for both binary and multi-class classification . It is flexible as we can select custom kernel as per the training data

SVM is independent of dimensionality of feature space. SVM uses overfitting protection , hence it can handle large number of features which are there in text classifiers

Balanced training set is in csr matrix format, which is used as independent variables and dependent variable is an array with equal dimensions for rows to that of training dataset


In [20]:
# from sklearn.svm import SVC

classifier = SVC(kernel= 'linear' , C =1 )
model = classifier.fit(x_res ,y_res) 
#tfs_test = vectorizer.transform(X_test)
feature_test = X_test
y_pred = classifier.predict(feature_test)

print(clsr(y_test.tolist(), y_pred.tolist()))

             precision    recall  f1-score   support

          0       0.95      0.94      0.95      1962
          1       0.66      0.69      0.68       316

avg / total       0.91      0.91      0.91      2278



In [17]:

tfs_test_1 = vectorizer.transform(List_strings_test)
y_pred_test = classifier.predict(tfs_test_1)

In [21]:
posts_test['Tag']=y_pred_test



In [23]:
posts_test.to_csv("Tagged_dataset_sentence_level.csv" , index= False)

In [40]:
y_pred = pd.DataFrame(y_pred)


0
0  0
1  0
2  0
3  0
4  0

### Running Naive Bayes model

The multinomial Naive Bayes classifier is suitable for classification with discrete features (e.g., word counts for text classification). The multinomial distribution normally requires integer feature counts. However, in practice, fractional counts such as tf-idf .

In [37]:
classifier = MultinomialNB()
model = classifier.fit(x_res ,y_res) 
#tfs_test = vectorizer.transform(X_test)
feature_test = X_test

from scipy.sparse import hstack
# feature_test = hstack((tfs_test,TempCount_test)).toarray()

y_pred_NB = classifier.predict(feature_test)

print(clsr(y_test.tolist(), y_pred_NB.tolist()))

             precision    recall  f1-score   support

          0       0.97      0.74      0.84      1962
          1       0.34      0.84      0.49       316

avg / total       0.88      0.75      0.79      2278



### Running KNearestNeighbour Model

kNN is powerful as it does not assume anything about the data, other than a distance measure can be calculated consistently between any two instances. It is non-parametric or non-linear as it does not assume a functional form.


In [38]:
classifier = KNeighborsClassifier()
model = classifier.fit(x_res ,y_res) 
#tfs_test = vectorizer.transform(X_test)
feature_test = X_test

from scipy.sparse import hstack
# feature_test = hstack((tfs_test,TempCount_test)).toarray()

y_pred_Knn = classifier.predict(feature_test)


print(clsr(y_test.tolist(), y_pred_Knn.tolist()))

             precision    recall  f1-score   support

          0       0.87      0.98      0.92      1962
          1       0.45      0.08      0.13       316

avg / total       0.81      0.86      0.81      2278



### K-Fold Cross Validation 

Cross Validation is a very useful technique for assessing the performance of machine learning models. It helps in knowing how the machine learning model would generalize to an independent data set.

K-fold is also a  model validation technique for assessing the effectiveness of the model, whether it is overfitting or underfitting. It is mainly used in settings where the goal is prediction, and one wants to estimate how accurately a predictive model will perform in practice . It randomly splits the entire dataset into k-folds. For each k folds in your dataset ,test the model on k – 1 folds of the data set. 

In [12]:
from sklearn.model_selection import KFold 
kf = KFold(n_splits=4)
for train_index, test_index in kf.split(X_1):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = x_res[train_index], x_res[test_index]
    y_train, y_test = y_res[train_index], y_res[test_index]
    classifier = SVC(kernel='linear' ,C = 1)
    model = classifier.fit(X_train ,y_train)
    feature_test = X_test
    y_pred = classifier.predict(feature_test)
    print(clsr(y_test.tolist(), y_pred.tolist()))

             precision    recall  f1-score   support

          0       0.99      0.92      0.95      1668
          1       0.61      0.92      0.73       230

avg / total       0.94      0.92      0.93      1898

             precision    recall  f1-score   support

          0       0.99      0.93      0.95      1615
          1       0.68      0.92      0.78       283

avg / total       0.94      0.92      0.93      1898

             precision    recall  f1-score   support

          0       0.98      0.93      0.95      1307
          1       0.86      0.95      0.90       591

avg / total       0.94      0.94      0.94      1898

             precision    recall  f1-score   support

          0       0.00      0.00      0.00         0
          1       1.00      0.90      0.95      1898

avg / total       1.00      0.90      0.95      1898



C:\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [152]:
Final_posts = pd.read_csv('Final_posts.csv')


## Checking post level accuracy
#### Step 1 : Creating training and testing splits

In [153]:
posts_sorted = posts.sort_values('Docs')
train = posts_sorted['Docs']<=1700
test = posts_sorted['Docs']>1700
training_se = posts_sorted[train]
test_se = posts_sorted[test]


#### Step 2 : Preprocessing and tokenizing into a list

In [154]:
words_se_level = preprocessing(training_se['Tokenized.Sentences'])

In [155]:
List_strings_se_level =[] 
for i in words_se_level:
    a = " ".join(str(x) for x in i)
    List_strings_se_level.append(a)


#### Step 3 : TF- IDF vectorizing

In [156]:
# Training data set creation
X_se_level = vectorizer.fit_transform(List_strings_se_level)
y_se_level = training_se['New_flag']

#### Step 4 : Balancing using SMOTE

In [157]:
sm_se = SMOTE(random_state=123,ratio = 1.0)
x_res_train, y_res_train = sm_se.fit_sample(X_se_level,y_se_level)


C:\Anaconda2\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)


#### Step 5 : Preprocessing test dataset

In [158]:
#Testing data creating - Preprocessing and Y_test creation
words_se_level_test = preprocessing(test_se['Tokenized.Sentences'])

List_strings_se_level_test =[] 
for i in words_se_level_test:
    a = " ".join(str(x) for x in i)
    List_strings_se_level_test.append(a)

  

In [159]:
X_se_test = vectorizer.transform(List_strings_se_level_test)
y_se_test = test_se['New_flag']  


In [160]:
x_res_train

<10846x7402 sparse matrix of type '<type 'numpy.float64'>'
	with 242167 stored elements in Compressed Sparse Row format>

#### Step 6 : Running Linear SVM model

In [161]:
classifier = SVC(kernel='linear' ,C =1)
model = classifier.fit(x_res_train ,y_res_train)   
#tfs_test = vectorizer.transform(X_test)
feature_test = X_se_test
from scipy.sparse import hstack
# feature_test = hstack((tfs_test,TempCount_test)).toarray()
y_pred = classifier.predict(feature_test)

print(clsr(y_se_test.tolist(), y_pred.tolist()))

             precision    recall  f1-score   support

          0       0.96      0.92      0.94      1129
          1       0.59      0.77      0.66       167

avg / total       0.92      0.90      0.91      1296



#### Step 6 : Appending predicted tag to test dataset

In [162]:
#appending predicted to test data set
test_se["predictions"] = y_pred

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


#### Step 7 : Creating post level data set from sentence level predicted dataset

In [163]:
tested_post_level = test_se.groupby('unique_key')['predictions'].agg(max)
test_dict = {'unique_key':tested_post_level.index.tolist(),'Predicted':tested_post_level.values}
test_df = pd.DataFrame(test_dict)

#### Step 8 : Merging dataset to calculate post level accuracy

In [164]:
post_level = pd.merge(Final_posts,test_df[['Predicted','unique_key']] ,on='unique_key')

In [165]:
y_test = post_level['TAG']
y_pred = post_level['Predicted']
pred = sum(post_level['Predicted'])
tag = sum(post_level['TAG'])
Precison = pred/tag
print Precison

0.8623188405797102
